In [13]:
!ls

CNN_dataset.zip


In [14]:
!unzip CNN_dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: CNN_dataset/CXR/009317.jpeg  
  inflating: __MACOSX/CNN_dataset/CXR/._009317.jpeg  
  inflating: CNN_dataset/CXR/002808.jpeg  
  inflating: __MACOSX/CNN_dataset/CXR/._002808.jpeg  
  inflating: CNN_dataset/CXR/000835.jpeg  
  inflating: __MACOSX/CNN_dataset/CXR/._000835.jpeg  
  inflating: CNN_dataset/CXR/001727.jpeg  
  inflating: __MACOSX/CNN_dataset/CXR/._001727.jpeg  
  inflating: CNN_dataset/CXR/004525.jpeg  
  inflating: __MACOSX/CNN_dataset/CXR/._004525.jpeg  
  inflating: CNN_dataset/CXR/006518.jpeg  
  inflating: __MACOSX/CNN_dataset/CXR/._006518.jpeg  
  inflating: CNN_dataset/CXR/008855.jpeg  
  inflating: __MACOSX/CNN_dataset/CXR/._008855.jpeg  
  inflating: CNN_dataset/CXR/009747.jpeg  
  inflating: __MACOSX/CNN_dataset/CXR/._009747.jpeg  
  inflating: CNN_dataset/CXR/001377.jpeg  
  inflating: __MACOSX/CNN_dataset/CXR/._001377.jpeg  
  inflating: CNN_dataset/CXR/004175.jpeg  
  inflating: __MACOSX/CNN_dataset

In [27]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [28]:
dataset = '/content/drive/MyDrive/mlas/CNN_dataset'
categories = ['HeadCT', 'ChestCT', 'AbdomenCT', 'Hand', 'CXR']
data = []
target = []

In [29]:
for i, category in enumerate(categories):
    path = os.path.join(dataset, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # convert to RGB
        image = cv2.resize(image, (32, 32)) # resize to (224, 224)
        data.append(image)
        target.append(i)

In [30]:
data = np.array(data)
target = np.array(target)

data, target = shuffle(data, target, random_state=42)

training, testing, training_target, testing_target = train_test_split(data, target, test_size=0.2, random_state=42)
training, validation, training_target, validation_target = train_test_split(training, training_target, test_size=0.2, random_state=42)

In [31]:
class_counts = np.bincount(training_target)
max_count = np.max(class_counts)
for i, count in enumerate(class_counts):
    if count != max_count:
        mask = training_target == i
        training = np.concatenate((training, training[mask]), axis=0)
        training_target = np.concatenate((training_target, training_target[mask]), axis=0)

training = training / 255.0
validation = validation / 255.0
testing = testing / 255.0

In [32]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(categories), activation='softmax')
])

In [33]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [37]:
modelLog = model.fit(training, training_target, validation_data=(validation, validation_target), epochs=10, batch_size=32)

Epoch 1/10
434/434 [==============================] - 3s 7ms/step - loss: 0.0027 - accuracy: 0.9994 - val_loss: 0.0055 - val_accuracy: 0.9988
Epoch 2/10
434/434 [==============================] - 2s 6ms/step - loss: 0.0020 - accuracy: 0.9996 - val_loss: 0.0036 - val_accuracy: 0.9992
Epoch 3/10
434/434 [==============================] - 2s 5ms/step - loss: 9.7904e-04 - accuracy: 0.9999 - val_loss: 0.0018 - val_accuracy: 0.9996
Epoch 4/10
434/434 [==============================] - 2s 5ms/step - loss: 0.0122 - accuracy: 0.9973 - val_loss: 0.0011 - val_accuracy: 0.9996
Epoch 5/10
434/434 [==============================] - 2s 5ms/step - loss: 0.0015 - accuracy: 0.9998 - val_loss: 1.3852e-04 - val_accuracy: 1.0000
Epoch 6/10
434/434 [==============================] - 3s 7ms/step - loss: 9.8989e-04 - accuracy: 0.9999 - val_loss: 4.1071e-05 - val_accuracy: 1.0000
Epoch 7/10
434/434 [==============================] - 3s 6ms/step - loss: 0.0017 - accuracy: 0.9999 - val_loss: 0.0045 - val_accurac

In [38]:
test_loss, test_acc = model.evaluate(testing, testing_target)
print('Test accuracy:', test_acc*100)

100/100 [==============================] - 0s 4ms/step - loss: 5.0005e-05 - accuracy: 1.0000
Test accuracy: 100.0


In [ ]:
# # Set the paths to the training, validation, and test directories
# train_dir = '/path/to/train_directory'
# val_dir = '/path/to/val_directory'
# test_dir = '/path/to/test_directory'

# # Define the data preprocessing pipeline
# train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
# val_datagen = ImageDataGenerator(rescale=1./255)
# test_datagen = ImageDataGenerator(rescale=1./255)

# # Load the training, validation, and test sets using the ImageDataGenerator class
# train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
# val_generator = val_datagen.flow_from_directory(val_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
# test_generator = test_datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')

# # Define the model architecture
# model = tf.keras.Sequential([
#     layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(128, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Flatten(),
#     layers.Dense(128, activation='relu'),
#     layers.Dense(5, activation='softmax')
# ])

# # Compile the model
# model.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# # Train the model on the training set and validate on the validation set
# model.fit(train_generator, epochs=10, validation_data=val_generator)

# # Evaluate the model on the test set
# test_loss, test_acc = model.evaluate(test_generator)
# print('Test accuracy:', test_acc)